<img src="https://github.com/nicholasmetherall/digital-earth-pacific-macblue-activities/blob/main/attachments/images/DE_Pacific_banner.JPG?raw=true" width="900"/>

Figure 1.1.a. Jupyter environment + Python notebooks

# Digital Earth Pacific Notebook 1 prepare postcard and load data to csv

The objective of this notebook is to prepare a geomad postcard for your AOI (masking, scaling and loading additional band ratios and spectral indices) and sampling all the datasets into a csv based on your training data geodataframe.

## Step 1.1: Configure the environment

In [1]:
from datetime import datetime
import numpy as np
import pandas as pd
import joblib
from sklearn.ensemble import RandomForestClassifier

In [2]:
# Predefined variable for title and version

# Enter your initials
initials = "nm"

# Date
date = datetime.now()

# Make a clean version string
version = f"{initials}-{date.strftime('%d%m%Y')}"
print(version)

nm-01092025


### Postcard csv

The objective of this notebook was to train the machine learning model that will allow us to classify an area with land cover classes defined through the training data.

Step 1.2. Input the training data to sample geomad data from the postcard

In [4]:
joined_df = pd.read_csv(f"training-data/{version}-joined_tdata.csv")
joined_df = joined_df.astype("float32")
joined_df

FileNotFoundError: [Errno 2] No such file or directory: 'training-data/nm-01092025-joined_tdata.csv'

In [4]:
# joined_df=joined_df.drop(columns=["y", "x"])

In [5]:
print(len(joined_df.columns))
joined_df.columns

35


Index(['cc_id', 'nir', 'red', 'blue', 'green', 'emad', 'smad', 'bcmad',
       'nir08', 'nir09', 'swir16', 'swir22', 'coastal', 'rededge1', 'rededge2',
       'rededge3', 'mndwi', 'ndti', 'cai', 'ndvi', 'evi', 'savi', 'ndwi',
       'b_g', 'b_r', 'mci', 'ndci', 'ln_bg', 'contrast', 'homogeneity',
       'energy', 'ASM', 'correlation', 'mean', 'entropy'],
      dtype='object')

In [ ]:
# Keep the dataframes as labelled dataframes, which makes things neater later
classes = joined_df["cc_id"]
observations = joined_df.drop(columns=["cc_id"])

# Create a model...
classifier = RandomForestClassifier()

# ...and fit it to the data
model = classifier.fit(observations, classes)

In [7]:
# Dynamically create the filename with f-string
file_path = f"models/{version}-test.model"

# Save the model
joblib.dump(model, file_path)

['models/nm-27072025-test.model']